In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import shutil
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import os
from torchvision import transforms
from transformers import BlipProcessor
from PIL import Image
from sklearn.model_selection import train_test_split

Mounted at /content/drive


Import the dfs

In [ ]:
one_hot_train_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/one_hot_train_df.csv')
one_hot_test_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/one_hot_test_df.csv')

w2v_train_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/w2v_train_df.csv')
w2v_test_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/w2v_test_df.csv')

# index_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/index_df.csv')
# w2v_df = pd.read_csv('/content/drive/MyDrive/Deep Final Work/Training dfs/w2v_df.csv')

In [ ]:
w2v_test_df.columns

Index(['ingredients', 'dish_calories', 'path', 'ingredient_embedding'], dtype='object')

In [ ]:
def split_dataframe(df, test_size=0.05, random_state=42):
    """
    Splits the DataFrame into training and testing sets.

    Args:
        df (pd.DataFrame): The DataFrame to split.
        test_size (float): Fraction of data to use as test set.
        random_state (int): Seed for reproducibility.

    Returns:
        train_df (pd.DataFrame): DataFrame for training.
        test_df (pd.DataFrame): DataFrame for testing.
    """
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

##Training over one-hot

### training the model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Image transformations for classification
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # Standard ImageNet normalization
])

# Custom Dataset Class
class IngredientDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]["path"]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        labels = torch.tensor(self.dataframe.iloc[idx, 1:].values.astype(float), dtype=torch.float32)

        return image, labels

one_hot_train_df = one_hot_train_df.drop(columns=['dish_calories'])
one_hot_test_df = one_hot_test_df.drop(columns=['dish_calories'])

train_dataset = IngredientDataset(one_hot_train_df, transform=transform)
test_dataset = IngredientDataset(one_hot_test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

image, labels = next(iter(train_loader))
print("Image shape:", image.shape)
print("Labels shape:", labels.shape)


Image shape: torch.Size([8, 3, 224, 224])
Labels shape: torch.Size([8, 198])


In [ ]:
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

# Load EfficientNet-B3 (pretrained)
model_onehot = models.efficientnet_b3(weights="IMAGENET1K_V1")

num_features = model_onehot.classifier[1].in_features
num_ingredients = one_hot_train_df.shape[1] - 1  # Exclude "path" column
model_onehot.classifier[1] = nn.Linear(num_features, num_ingredients)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_onehot = model_onehot.to(device)


Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 128MB/s]


In [ ]:
import torch

# Initialize the model
# Count the number of trainable parameters (degrees of freedom)
dof = sum(p.numel() for p in model_onehot.parameters() if p.requires_grad)

print(f"Degrees of Freedom (Trainable Parameters): {dof}")

Degrees of Freedom (Trainable Parameters): 11000558


In [ ]:

# Binary Cross-Entropy with Logits (applies sigmoid internally)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_onehot.parameters(), lr=1e-4)
num_epochs = 10

for epoch in range(num_epochs):
    # ----- Training Phase -----
    model_onehot.train()
    total_train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_onehot(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")

    # ----- Evaluation Phase -----
    model_onehot.eval()
    total_test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()

            correct += (preds == labels).sum().item()
            total += labels.numel()

    avg_test_loss = total_test_loss / len(test_loader)
    test_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Test Loss: {avg_test_loss:.4f} - Test Accuracy: {test_accuracy:.4f}")

# Save trained model
torch.save(model_onehot.state_dict(), "/content/drive/MyDrive/Deep Final Work/Saved Models/efficient_3_one_hot.pth")
print("✅ Model training complete and saved!")


Epoch 1/10 - Train Loss: 0.1567
Epoch 1/10 - Test Loss: 0.0981 - Test Accuracy: 0.9714
Epoch 2/10 - Train Loss: 0.0862
Epoch 2/10 - Test Loss: 0.0824 - Test Accuracy: 0.9732
Epoch 3/10 - Train Loss: 0.0761
Epoch 3/10 - Test Loss: 0.0733 - Test Accuracy: 0.9756
Epoch 4/10 - Train Loss: 0.0673
Epoch 4/10 - Test Loss: 0.0659 - Test Accuracy: 0.9768
Epoch 5/10 - Train Loss: 0.0600
Epoch 5/10 - Test Loss: 0.0602 - Test Accuracy: 0.9794
Epoch 6/10 - Train Loss: 0.0533
Epoch 6/10 - Test Loss: 0.0552 - Test Accuracy: 0.9807
Epoch 7/10 - Train Loss: 0.0477
Epoch 7/10 - Test Loss: 0.0517 - Test Accuracy: 0.9823
Epoch 8/10 - Train Loss: 0.0412
Epoch 8/10 - Test Loss: 0.0495 - Test Accuracy: 0.9840
Epoch 9/10 - Train Loss: 0.0366
Epoch 9/10 - Test Loss: 0.0469 - Test Accuracy: 0.9839
Epoch 10/10 - Train Loss: 0.0321
Epoch 10/10 - Test Loss: 0.0528 - Test Accuracy: 0.9848
✅ Model training complete and saved!


## Train with W2V embedding #Not used  

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import ast

# -------------------------------
# 1. Data Preparation: Define a Dataset for W2V Embeddings
# -------------------------------
class IngredientEmbeddingDataset(Dataset):
    def __init__(self, dataframe, transform=None, embedding_col='ingredient_embedding'):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame with columns 'path' and a column (by default 'ingredient_embedding')
                                     containing the precomputed W2V embedding (as a list or string).
            transform: Image transformations.
            embedding_col (str): Column name for the target embedding.
        """
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        self.embedding_col = embedding_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['path']).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # Retrieve the precomputed W2V embedding.
        embedding_data = row[self.embedding_col]
        if isinstance(embedding_data, str):
            if ',' in embedding_data:
                embedding_data = ast.literal_eval(embedding_data)
            else:
                embedding_data = embedding_data.strip("[]")
                embedding_data = [float(x) for x in embedding_data.split()]
        target_embedding = torch.tensor(embedding_data, dtype=torch.float)
        return image, target_embedding

# -------------------------------
# 2. Define Image Transformations
# -------------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # Standard ImageNet normalization
])


# -------------------------------
# 3. Create Dataset and DataLoaders
# -------------------------------
train_dataset = IngredientEmbeddingDataset(w2v_train_df, transform=transform, embedding_col='ingredient_embedding')
test_dataset  = IngredientEmbeddingDataset(w2v_test_df, transform=transform, embedding_col='ingredient_embedding')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=8, shuffle=False)

# -------------------------------
# 4. Modify the Model to Output W2V Embeddings
# -------------------------------
# Load EfficientNet-B3 (pretrained)
model = models.efficientnet_b3(weights="IMAGENET1K_V1")

# Replace the final classifier layer to output a vector with dimension equal to the W2V embedding dimension.
num_features = model.classifier[1].in_features
w2v_dim = 50
model.classifier[1] = nn.Linear(num_features, w2v_dim)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# -------------------------------
# 5. Define Loss Function and Optimizer
# -------------------------------

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

# -------------------------------
# 6. Training Loop
# -------------------------------
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    for images, target_embedding in train_loader:
        images = images.to(device)
        target_embedding = target_embedding.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, target_embedding)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item() * images.size(0)

    avg_train_loss = total_train_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")

    model.eval()
    total_test_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_test_loss += loss.item()

            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()

            correct += (preds ==
                        ).sum().item()
            total += labels.numel()

    avg_test_loss = total_test_loss / len(test_loader)
    test_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Test Loss: {avg_test_loss:.4f} - Test Accuracy: {test_accuracy:.4f}")

# Save the trained model.
torch.save(model.state_dict(), "/content/drive/MyDrive/Deep Final Work/Saved Models/efficient_b3_w2v.pth")
print("✅ Model training complete and saved!")


Epoch 1/10 - Train Loss: 0.0109
Epoch 1/10 - Test Loss: 0.0027 - Test Accuracy: 0.7863
Epoch 2/10 - Train Loss: 0.0031
Epoch 2/10 - Test Loss: 0.0013 - Test Accuracy: 0.6543
Epoch 3/10 - Train Loss: 0.0021
Epoch 3/10 - Test Loss: 0.0010 - Test Accuracy: 0.5603
Epoch 4/10 - Train Loss: 0.0016
Epoch 4/10 - Test Loss: 0.0011 - Test Accuracy: 0.4899
Epoch 5/10 - Train Loss: 0.0013
Epoch 5/10 - Test Loss: 0.0007 - Test Accuracy: 0.4353
Epoch 6/10 - Train Loss: 0.0011
Epoch 6/10 - Test Loss: 0.0007 - Test Accuracy: 0.3916
Epoch 7/10 - Train Loss: 0.0009
Epoch 7/10 - Test Loss: 0.0007 - Test Accuracy: 0.3558
Epoch 8/10 - Train Loss: 0.0008
Epoch 8/10 - Test Loss: 0.0007 - Test Accuracy: 0.3261
Epoch 9/10 - Train Loss: 0.0007
Epoch 9/10 - Test Loss: 0.0006 - Test Accuracy: 0.3009
Epoch 10/10 - Train Loss: 0.0006
Epoch 10/10 - Test Loss: 0.0005 - Test Accuracy: 0.2794
✅ Model training complete and saved!
